## Introduction 

This analysis focuses on predicting an individual's income level using machine learning modeling techniques. By employing methods like Naive Bayes, Neural Networks, and Decision Trees, we aim to understand how various features like marital status or occupation influence whether a person falls into a higher or lower income bracket.  Specifically, our goal is to develop a model capable of predicting whether an individual earns over or under $50,000 annually, which could have numerous applications, from targeted marketing strategies to socio-economic research.

In [50]:
# Import needed packages and potential data science tools like pandas and numpy
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.compose import make_column_selector, ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from xgboost import XGBClassifier
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier, StackingClassifier
from sklearn.linear_model import Ridge
from sklearn.metrics import *
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from imblearn.pipeline import Pipeline as ImblearnPipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPRegressor,MLPClassifier
from sklearn.naive_bayes import CategoricalNB, ComplementNB
import warnings

## Data Preparation

Before we can apply machine learning models, it's crucial to prepare our data. This involves tasks like handling missing values, encoding categorical variables, and normalizing features. These steps ensure that our models receive clean and structured data, which leads to more accurate predictions.  In this case, I identified several differing data types, including objects and integers, that needed to be addressed before modeling. I also resolved any instances of special characters (e.g., ?) in the data to prevent noise and avoid potential errors. The most important step was transforming the Income column into a binary (dummified) column, where a value of 1 indicates an income greater than 50,000, and 0 indicates an income of 50,000 or less. This transformation simplified the preprocessing required later on.

In [57]:
# Read-in the data, naming our df purposefully
income = pd.read_csv("/Users/rileysvensson/Desktop/GSB 545 - Advanced Machine Learning/data /income_evaluation.csv")

In [58]:
# Remove leading and trailing spaces from column names, like " income"
income.columns = income.columns.str.strip()

# Dummify our target variable to be 1 or 0, 1 for people who make over 50k in income
income['Income_Over50'] = (income['income'] == ' >50K' ).astype(int)

# Ensure that the dummification worked to keep same distribution of >50k income's
income['Income_Over50'].value_counts()

# Both value count functions display the same 24,720 people making below 50k compared to 7,841 making above
income['income'].value_counts()

# Check to ensure that all the variable datatypes are as expected (numerical columns are Int, categorical are Object)
income.dtypes

# Check for missing values before modeling, displaying no issues 
income.isnull().sum()


age               0
workclass         0
fnlwgt            0
education         0
education-num     0
marital-status    0
occupation        0
relationship      0
race              0
sex               0
capital-gain      0
capital-loss      0
hours-per-week    0
native-country    0
income            0
Income_Over50     0
dtype: int64

In [59]:
# Drop duplicate target column
income_final = income.drop(['income'], axis = 1)

# Replace '?' with 'Unknown' in the all column's
income_final['workclass'] = income_final['workclass'].replace(' ?', 'Unknown')
income_final['occupation'] = income_final['occupation'].replace(' ?', 'Unknown')
income_final['native-country'] = income_final['native-country'].replace(' ?', 'Unknown')

# Make the ?'s into Unknown for workclass just in case the model cannot handle it
income_final['workclass'] = income_final['workclass'].replace('?', 'Unknown')

# View final dataset before modeling to ensure data is prepared
income_final.head(5)

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,Income_Over50
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,0
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,0
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,0
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,0
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,0


#### Before modeling, we must ask ourselves a few questions:
1. Do we need all predictors? For example, both education and education-num, or does one provide enough predictive power without the other?
2. Is each fnlwgt's a unique value that doesn't provide predictability, or are these values simply a continous spectrum?
3. What is the best options of modeling methods in this case, besides Naive Bayes and Neural Networks.
4. What scoring metrics are appropriate for model validation?

To test the predictability of the education feature's we will have to run identical model types including each variable without the other. And since education and education-num are perfectly collinear (meaning they convey the same information), we cannot include both in our models.  Also, after looking to the data dictionary, `fnlwgt` is labeled as a continuous variable numerically representing an individual's education which provided no predictive power to our model.  As for the last point, we know that **XGBoost can be used to deal with imbalanced target variables**, as we have here with only 25% of the people making over 50k, indicating it may be a good use case.  Also, **creating a stacked model that combines the predictability of several models could prove successful**.  Lastly, it our options for scoring metrics include the classification associated scores like accuracy, precision, recall and f1-score.  Like mentioned earlier, it is typically advised to avoid accuracy when faced with unbalanced distribution of our target class, while precision and recall typically are used in specific scenarios in which we need to more accuratley predict the target, for example when working with fraud or cancer testing data.  In this case, **f1 is a more hollistic option, which will display a tradeoff between recall and precision, so what % we are able to accurately predict of people making over 50k, and at what success rate.**


### Feature Selection & Preprocessing

In [60]:
# Define X and Y sets, dropping relationship results in better scoring
X = income_final.drop(["Income_Over50",'relationship','sex','education','fnlwgt'], axis = 1)
y = income_final["Income_Over50"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 42)

# Define column transformer for preprocessing models like NeuralNetworks, XgBoost
ct_all = ColumnTransformer(
  [
    ("dummify", OneHotEncoder(sparse_output = False, handle_unknown='ignore'),
    make_column_selector(dtype_include=object)),
    ("standardize", StandardScaler(), make_column_selector(dtype_include=np.int64))
  ],
  remainder = "passthrough" 
).set_output(transform = "pandas")

# Define a column transformer to ensure all X's are encoded to be between 0 and 1, to remove -1 input error for NB
ct_nb = ColumnTransformer(
    [
        ("dummify", OneHotEncoder(sparse_output=False, handle_unknown='ignore'),
        make_column_selector(dtype_include=object)),
        ("scale_to_non_negative", MinMaxScaler(), make_column_selector(dtype_include=np.number))
    ],
    remainder='passthrough' 
)

## Model 1: Naive Bayes (Categorical)

Naive Bayes is a simple yet effective classification algorithm, meaning that it is only applied to categorical data. It operates under the 'naive' statistical assumption that features are independent of one another, which simplifies calculations and results in fast predictions. Despite its simplicity, Naive Bayes often performs well on real-world datasets.

In [30]:
# NB Pipeline
naive_bayes_pipeline = Pipeline([
    ("preprocessor", ct_nb),
    ("naivebayesclassifier", CategoricalNB())
])

# Define the parameter grid for CategoricalNB
param_grid = {
    'naivebayesclassifier__alpha': [1.0, 0.5, 0.1, 0.01],  
    'naivebayesclassifier__min_categories': [1, 2, 4, None],
    'naivebayesclassifier__fit_prior': [True, False]
}

# Tuning with GridSearchCV, using F1 score as the scoring metric for classification
grid_search = GridSearchCV(naive_bayes_pipeline, param_grid, cv=5, scoring='f1_macro')

grid_search.fit(X_train, y_train)

# Best parameters and score
print("Best parameters: ", grid_search.best_params_)

# Predict on the test set
test_predictions = grid_search.predict(X_test)
test_f1 = f1_score(y_test, test_predictions, average='macro')

print("\n Naive Bayes - F1 Score: ", test_f1)

Best parameters:  {'naivebayesclassifier__alpha': 0.01, 'naivebayesclassifier__fit_prior': True, 'naivebayesclassifier__min_categories': 1}

 Naive Bayes - F1 Score:  0.726688402425609


Our Naive Bayes Classifier, was able to predict the dataset moderately well with an F1-score of 0.7266.  This means **that our model can accurately predict around 73% of the people who make over 50k in Income**. As a general guideline, an F1 score of 0.7 or higher is often considered good, so we are right on this threshold.  Using the categorical NB classifier was the proper choice here, as our target variable and all variables can be thought of as categories, besides maybe final_weights or `ages`. I ran a few NB model's, removing `education` and `education-num`, and discovered a clear decline in predictability of our model, suggesting that both are needed in our feature set, and that education has a positive impact on Income. Based on intuition I also iterated through a few NB model's with different feature sets, finding that removing `relationship` and `sex` from our model provides more predictability from 0.7 to 0.7266.  This makes conceptual sense, as once's relationship status could contribute to their Income, however it shouldn't really as your income is an indivdually-earned and associated value. And of course, sexual discrimination is not allowed in the workplace so someone's sex should not impact the amount of money they earn, confirmed by our model.

## Model 2: Neural Networks MLPClassifier

In our second attempt, we leveraged a Neural Network model, which are powerful models capable of capturing complex relationships in the data by mimicking the way our brain detects patterns. They consist of layers of interconnected nodes, or 'neurons,' that work together to learn patterns. While they require more computational resources than simpler models like Naive Bayes, they often achieve higher accuracy, particularly when working with large datasets with constant information to teach our model what traits attribute to a person making over 50k a year.

In [32]:
# MLP Pipeline - 1
neural_network = Pipeline([
    ("preprocessor", ct_all),
    ("mlpclassifier", MLPClassifier(max_iter = 1000, early_stopping = True))
])

# Define the parameter grid for tuning, including solver, 1 hidden layer with 100 units, learning rate and activation functions
param_grid = {
    'mlpclassifier__hidden_layer_sizes': [(100,)],  
    'mlpclassifier__activation': ['relu', 'tanh', 'logistic','identity'],
    'mlpclassifier__solver':['adam','sgd'],                               # lbfgs resulted in failure to converge
    'mlpclassifier__learning_rate': ['constant','invscaling','adaptive']
}

# Tuning with GridSearchCV, using r2 as the scoring metric for classification
grid_search = GridSearchCV(neural_network, param_grid, cv=5, scoring='f1_macro')

grid_search.fit(X_train, y_train)

# Best parameters and score
print("Best parameters: ", grid_search.best_params_)

# Predict on the test set
test_predictions_nn = grid_search.predict(X_test)
test_f1_nn = f1_score(y_test, test_predictions_nn, average='macro')

print("\n Neural Networks - F1 Score: ", test_f1_nn)

Best parameters:  {'mlpclassifier__activation': 'relu', 'mlpclassifier__hidden_layer_sizes': (100,), 'mlpclassifier__learning_rate': 'adaptive', 'mlpclassifier__solver': 'adam'}

 Neural Networks - F1 Score:  0.7996550233033553


Our Neural Network Classifier, showed significant improvement in its ability to predict those making over 50 thousand dollars, with a new F1-score of nearly 0.80. When referring to our general guideline, **an F1 score of 0.8 is classified as great, meaning that we are predicting a large % of the target class correctly, and of those predicted to make over 50k, we are not receiving many false positives or untrue predictions.**  In terms of the tuned parameters, the 'adam' solver is used as the default within MLP as it typically results in the best predictions, which makes sense why it was our best solver.  The rectified linear unit was our optimal activation function,  suggesting that the data may follow a pattern in which a specific age or combination of predictors, results in an increase or decrease in Income or those making 50k.  From the previous NN model's I have ran, the 'constant' learning rate is best paired with the adam solver, with some options like adaptive, only being applicable for specific situations.  The best model capable of running quickly include one hidden layer, with 100 hidden units, that was able to adjust our weights and accurately predict those making over 50k.

## Model 3: XGBoosting Tree

In our final model, we applied XGBoosting, which stands for Extreme Gradient Boosting. XGBoost is a highly efficient and powerful implementation of the gradient boosting algorithm, known for its ability to handle large datasets and complex data patterns. It works by creating an ensemble of decision trees, where each subsequent tree aims to correct the errors of the previous ones, enhancing the model's predictive accuracy. XGBoost is often the go-to choice for machine learning tasks due to its robustness and flexibility, making it a fitting choice for our imbalanced classification problem.

In [72]:
# Define a grid of possible weight values that are close to distribution of imbalance target 24,720 / 7,841 = 3.15
pos_weight_values = [1.30, 1.305]
    
# Setup pipeline #1 for XGBoost
xgb_pipeline = Pipeline([
    ("preprocessor", ct_all),
    ("classifier", XGBClassifier(use_label_encoder=False, eval_metric='logloss'))  # Parameters needed to suppress warnings
])

# Parameter grid for XGBoost with different parameters being tuned
param_grid_xgb = {
    'classifier__n_estimators': [100,200,300],             
    'classifier__learning_rate': [0.1, 0.3, 0.5],    
    'classifier__max_depth': [3,4],                  
    'classifier__scale_pos_weight': pos_weight_values
}

# Setup GridSearchCV for XGBoost                               # n_jobs -1 tells Jupyter to user all CPU's for gridsearch
grid_search_xgb = GridSearchCV(xgb_pipeline, param_grid_xgb, cv=5, scoring='f1_macro', n_jobs=-1)

# Fitting our model on training data
grid_search_xgb.fit(X_train, y_train)

# Best parameters and score
print("Best parameters: ", grid_search_xgb.best_params_)

# Predict on the test set
test_xgb_predictions = grid_search_xgb.predict(X_test)

# Use f1_score function
test_f1_xgb = f1_score(y_test, test_xgb_predictions)

print("\n XGBoost - F1 Score: ", test_f1_xgb)

Best parameters:  {'classifier__learning_rate': 0.1, 'classifier__max_depth': 4, 'classifier__n_estimators': 200, 'classifier__scale_pos_weight': 1.305}

 XGBoost - F1 Score:  0.7280779450841453


The Extreme Gradient Boosting Model (XGBoost) surprisingly did not outperform the other models in this analysis, achieving an F1 score of 0.728. While the inclusion of the `scale_pos_weight parameter`, which adjusts for class imbalance, was intended to maximize the trade-off between precision and recall, it did not result in the highest performance. In this case, the parameter was set to 1.305, reflecting the ratio of individuals earning less than 50k to those earning more. This ratio, being above 1, indicates an imbalanced class problem, which is a challenge that XGBoost typically handles well.
The other best parameters identified for the XGBoost model were a `learning_rate` of 0.1,`max_depth` of 4, and `n_estimators` of 300. However, despite these settings, the XGBoost model's performance was on par with the Naive Bayes model and did not exceed the performance of the Neural Network, which achieved a higher F1 score. This outcome suggests that **while XGBoost is a powerful and flexible model, in this particular income classification task, it did not outperform the simpler models or the more complex Neural Network**.

## Model 4: Stacked Classifier
Neural Network, XgBoost, Naive Bayes


In [63]:
# Define base models within the preprocessing pipeline, and tuning with optimal values from Models 1-3

base_models = [
    ('gradient_boosting', Pipeline([
        ('preprocessor', ct_all),
        ('classifier', GradientBoostingClassifier(learning_rate=0.1, max_depth=4, n_estimators=300))
    ])),
    ('xgbboost', Pipeline([
        ('preprocessor', ct_all),
        ('classifier', XGBClassifier(learning_rate=0.1, max_depth=4, n_estimators=300,
                                                  scale_pos_weight=1.305))
    ])),
    ('mlp_classifier', Pipeline([
        ('preprocessor', ct_all),
        ('classifier', MLPClassifier(activation='relu', hidden_layer_sizes=(100,), 
                                     learning_rate='constant', solver='adam', max_iter=1000))
    ])),
    ('categorical_nb', Pipeline([
        ('preprocessor', ct_nb),
        ('classifier', CategoricalNB(alpha=0.5, fit_prior=True, min_categories=1))
    ]))
]

In [64]:
# Define the final regressor method, typically Linear or Logistic, in this case for classification use Log
final_regressor = LogisticRegression()

# Create the stacking ensemble
stacking_classifier = StackingClassifier(estimators=base_models, final_estimator=final_regressor, cv=5)

# Fit the stacking model on train data
stacking_classifier.fit(X_train, y_train)

# Predict/evaluate the model on Test data
stacking_predictions = stacking_classifier.predict(X_test)
stacking_f1 = f1_score(y_test, stacking_predictions, average = 'macro')

# Stacking model performed best 
print("Stacking Model - F1 Score: ", stacking_f1)

Stacking Model - F1 Score:  0.8199020389171616


## Conclusion 

Finally by utilizing our new method of a stacked model, which includes iterative learning from multiple selected models, we were able to **increase our F1 to be 0.822**, to be above the threshold of 0.8 and **indicating that our model both accurately and precisely predicts those making over 50k a year**.  To achieve the best results, we combined the predictive power of our XGBoosting Tree, Categorical Naive Bayes, Neural Network, and Gradient Boosting model to maximize our performance metric, F1.  We settled upon a simple yet powerful final feature set, consisting of the following variables used to predict if Income is greater than 50 thousand or not:

1. age	
2. workclass	
3. fnlwgt	
4. education	
5. education-num	
6. marital-status
7. occupation	
8. race	            
9. capital-gain	
10. capital-loss	
11. hours-per-week	
12. native-country

In terms of some specific effects of these variables on income, it can be intuitively confirmed that variables like working hours-per-week, education, and capital gain, have a strongly positive relationship with income.  While on the other hand, capital loss likely has a strong negative association with income.  However, the remaining predictors specifically like Age and Education, differ based on each person and their situations, resulting in unclear and more complex effects on income.  For example, some millionaires may not have gone to college and reach this wealth at a young age, which makes predicting income a bit more complex than one may assume before diving into some data.  

As for a next step, it would be useful to find a few additional predictors to include in this dataset, potentially related to their workplace attitude or personality.  It would also be beneficial to dive further into the effects of each complex feature to derive richer insights, such as why race has any effect on income, as this is discriminatory and should not be seen in data.  The same would be interesting to know about other variables such as, marital-status.  For instance, does being married contribute to more wealth if spouse's share revenue, or does it make for additional costs associated with the relationship?  However, for now we are able to accurately predict Income (>50k) with a final F1 score of 87., and use this model in segmentation, targeted marketing, and other financial use cases.